外場は与えず、原子間斥力のみ与えて Quench して原子配置を最適化する。


In [1]:
from logging import getLogger

import numpy as np

from graphenator.quench import quench_particles
from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force

logger = getLogger()

# 斥力で配置を調整する原子の個数。最終的に配置する炭素の数ではない。
Natom = 192
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 初期配置。セル相対。
r = random_box(Natom)

# ポテンシャルエネルギーは粒子位置の関数。
pot = lambda r, cell: repulsive_potential(r, cell, repul=4, rc=2.0)
# その勾配。
dpot = lambda r, cell: -repulsive_force(r, cell, repul=4, rc=2.0)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 4514.986982
         Iterations: 9
         Function evaluations: 249
         Gradient evaluations: 235


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning: Desired error not necessarily achieved due to precision loss.
  res = _minimize_cg(f, x0, args, fprime, callback=callback, c1=c1, c2=c2,


Quench 前とあとを比較。


In [2]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go
from itertools import starmap


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        go.Scatter3d(
            x=x[:, 0],
            y=x[:, 1],
            z=x[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

外場を加えたエネルギー関数とその傾きの関数を準備する。


In [4]:
# The gyroid surface and its gradient
from logging import getLogger, basicConfig, INFO

import numpy as np

from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force

from graphenator.surface import Gyroid, Sphere, Diamond, PSurface

# from graphenator.surface.psurface import PSurface


basicConfig(level=INFO)
logger = getLogger()

# 斥力粒子の個数
Natom = 192
# 外場の強度。小さいと、ぎゅうぎゅうづめになった時に面外に粒子がこぼれる可能性がある。
cost = 100000
# 立方体セル
cell = np.diag(np.array([5.0, 5.0, 5.0]))

# 曲面関数
# surf = Gyroid(eccentricity=0.0)
# surf = Sphere(radius=0.45)
# surf = Diamond(eccentricity=0.0)
surf = PSurface(eccentricity=0.0)

# 斥力粒子の座標(セル相対)
r = random_box(Natom)

# 斥力粒子のポテンシャルエネルギー関数
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
# その勾配
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

Quench し、表示する。


In [5]:
from graphenator.quench import quench_particles
import plotly.graph_objects as go

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()

         Current function value: 3861713.440088
         Iterations: 4
         Function evaluations: 60
         Gradient evaluations: 50


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



Quench した構造に有限温度を与えてすこし原子が動けるようにし、100 ステップの疑似分子動力学法を実施する。これにより、より歪みが少ない形状にたどりつけるはず。

斥力粒子を均質に配置して三角格子をつくったら、三角格子の重心同士を連結して、双対グラフを生成する。これがグラフェンの構造となる。一連の処理は graphenate 関数の内部で行われる。

ここでは結果を yaplot 形式で出力している。


In [6]:
from graphenator import draw_yaplot, graphenate

with open(f"psurface.yap", "w") as file:
    count = 100
    for x, cell, g in graphenate(
        Natom,
        cell,
        pot,
        dpot,
        dt=0.0005,  # 0.005
        T=0.1,
    ):
        print(count)
        file.write(draw_yaplot(x, cell, g))
        count -= 1
        if count == 0:
            break

/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.

INFO:root:packing candid.
INFO:root:[0, 0, 0, 298, 38, 6, 0] Cycles in the packing
INFO:root:Tempering


         Current function value: 21017.491178
         Iterations: 158
         Function evaluations: 594
         Gradient evaluations: 578


INFO:root:packing candid.
INFO:root:[0, 0, 0, 298, 38, 6, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 298, 38, 6, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 294, 40, 6, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 293, 42, 5, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 295, 41, 5, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 295, 41, 5, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 302, 39, 4, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 309, 37, 3, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 311, 36, 3, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 311, 36, 3, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 309, 37, 3, 0] Cycles in the packing
INFO:root:packing candid.
INFO:root:[0, 0, 0, 307, 38, 3, 0] Cycles in the packing
INFO

100


INFO:root:[0, 0, 0, 310, 41, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 18, 132, 42, 0] Cycles in the graph
INFO:root:packing candid.


...

3


INFO:root:packing candid.
INFO:root:[0, 0, 0, 392, 0, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 5, 158, 29, 0] Cycles in the graph
INFO:root:packing candid.


2


INFO:root:[0, 0, 0, 392, 0, 0, 0] Cycles in the packing
INFO:root:Quenching the graph...
INFO:root:Done.
INFO:root:[0, 0, 0, 0, 0, 5, 158, 29, 0] Cycles in the graph


1


曲面を解析的な式ではなく、グリッド上の数値配列として与える。

`cube`や`sphere`は 3 次元グリッドで定義される$w=f(x,y,z)$型の空間関数で、これが$f(x,y,z)=0$を横切る isosurface で、graphene の形状を指定する。


In [13]:
from logging import INFO, basicConfig, getLogger, DEBUG

import numpy as np

from graphenator.quench import quench_particles
from graphenator.pack import random_box
from graphenator.interaction import repulsive_potential, repulsive_force
from graphenator.surface import Grid, Ticks, GridSurfaceFunc


basicConfig(level=INFO)
logger = getLogger()

Natom = 192
cost = 10000
cell = np.diag(np.array([5.0, 5.0, 5.0]))

r = 0.4  # "radius" of the cube, i.e., half length of an edge

XYZ = np.mgrid[-0.5:0.5:11j, -0.5:0.5:11j, -0.5:0.5:11j]
xticks = Ticks(min=-0.5, binw=0.1)
yticks = Ticks(min=-0.5, binw=0.1)
zticks = Ticks(min=-0.5, binw=0.1)

# グリッドタイプの関数定義法
cube = Grid(
    values=np.max(np.abs(XYZ), axis=0) - r, xticks=xticks, yticks=yticks, zticks=zticks
)

sphere = Grid(
    values=np.sum(XYZ**2, axis=0) - r * r,
    xticks=xticks,
    yticks=yticks,
    zticks=zticks,
)

surf = GridSurfaceFunc(sphere)

In [14]:
r = random_box(Natom)
# x = r @ cell
pot = lambda r, cell: repulsive_potential(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_potential(r)
dpot = lambda r, cell: -repulsive_force(
    r, cell, repul=4, rc=1.0
) + cost * surf.exfield_gradient(r)

r_quenched = quench_particles(r, cell, pot, dpot)
x_quenched = r_quenched @ cell

         Current function value: 19660.800824
         Iterations: 15
         Function evaluations: 215
         Gradient evaluations: 203


/Users/matto/.local/share/virtualenvs/gyroid-cqmDCkE1/lib/python3.11/site-packages/scipy/optimize/_optimize.py:1764: OptimizeWarning:

Desired error not necessarily achieved due to precision loss.



In [15]:
# -*- coding: utf-8 -*-
import plotly.graph_objects as go


# mode='makers'を指定しないと点の間に線が引かれる
fig = go.Figure(
    data=[
        # go.Scatter3d(
        #     x=x[:, 0],
        #     y=x[:, 1],
        #     z=x[:, 2],
        #     mode="markers",
        #     marker=dict(
        #         size=6,
        #         # color=point_sum,
        #         colorscale="Viridis",
        #         # symbol=result_symbol
        #     ),
        # ),
        go.Scatter3d(
            x=x_quenched[:, 0],
            y=x_quenched[:, 1],
            z=x_quenched[:, 2],
            mode="markers",
            marker=dict(
                size=5,
                # color=point_sum,
                colorscale="Viridis",
                # symbol=result_symbol
            ),
        ),
    ]
)
fig.show()